In [1]:
# data: http://share.mailcharts.com/0X2N2L2x3p47

import pandas as pd
import numpy as np
import math
import re
import time

In [9]:
def combine_categories_set(row):
    result = row.category_1.lower()
    
    if row.category_2:
        result = result + " " + row.category_2.lower()
    if row.category_3:
        result = result + " " + row.category_3.lower()
    if row.category_4:
        result = result + " " + row.category_4.lower()
    if row.category_5:
        result = result + " " + row.category_5.lower()
    if row.category_6:
        result = result + " " + row.category_6.lower()
    if row.category_7:
        result = result + " " + row.category_7.lower()
    
    result = set(re.split(r"[\>|\&| ]+", result))
    return result

def combine_categories(row):
    result = row.category_1.lower()
    
    if row.category_2:
        result = result + " > " + row.category_2.lower()
    if row.category_3:
        result = result + " > " + row.category_3.lower()
    if row.category_4:
        result = result + " > " + row.category_4.lower()
    if row.category_5:
        result = result + " > " + row.category_5.lower()
    if row.category_6:
        result = result + " > " + row.category_6.lower()
    if row.category_7:
        result = result + " > " + row.category_7.lower()

    return result

In [10]:
# https://www.google.com/basepages/producttype/taxonomy.en-US.txt
cat = pd.read_csv("./data/taxonomy.csv", encoding="ISO-8859-1")
cat.fillna(False, inplace=True)
cat["combined"] = cat.apply(lambda x: combine_categories(x), axis=1)
cat["combined_set"] = cat.apply(lambda x: combine_categories_set(x), axis=1)
print(cat.shape)
cat.head(2)

(5427, 9)


,category_1,category_2,category_3,category_4,category_5,category_6,category_7,combined,combined_set
0,Animals & Pet Supplies,False,False,False,False,False,False,animals & pet supplies,"{animals, pet, supplies}"
1,Animals & Pet Supplies,Live Animals,False,False,False,False,False,animals & pet supplies > live animals,"{animals, pet, supplies, live}"


In [11]:
df = pd.read_csv("./data/emails.csv", encoding="ISO-8859-1")
df = df.iloc[:50]
print(df.shape)
df.head(2)

(50, 2)


,subject,full_text
0,Fine-tune your collection with Edox + Versace,"PLUS, enter our Invicta Big Idea Sweepstakes f..."
1,Des Moines Register Evening Report,Having trouble viewing this email? | View it i...


In [12]:
# def contains_word(s, w):
#     return (' ' + w + ' ') in (' ' + s + ' ')

# subjects = df.iloc[0:10].subject.apply(lambda x: x.lower())

# start_time = time.time()
# categories = []

# for subject in subjects:
#     category = set()
#     for i, row in cat.iterrows():
#         words = row["combined_set"]
#         for w in words:
#             if contains_word(subject, w):
#                 category.add(i)
#     categories.append(list(category))

# print("--- %s seconds ---" % (time.time() - start_time))
# # --- 3.7804958820343018 seconds ---
# # df.categories = categories

In [13]:
subjects = df.subject.apply(lambda x: x.lower())

start_time = time.time()
categories = []

for subject in subjects:
    category = set()
    subject = set(re.split(r"[\>|\&| ]+", subject))

    for i, row in cat.iterrows():
        words = row["combined_set"]
        
        if len(words.intersection(subject))>0: 
            category.add(i)

    categories.append(list(category))

print("--- %s seconds ---" % (time.time() - start_time))
# --- 3.6054348945617676 seconds ---
df["category_ids"] = categories

--- 74.1706759929657 seconds ---


In [14]:
# def print_categories(list_of_ids):
#     for i in list_of_ids:
#         category = cat.iloc[i, :-1]
#         print(" > ".join([i for i in category if i != False]))
        
# print_categories(list(category))

In [15]:
def convert_categories_index_to_text(categories):
    result = []
    for i in categories:
        result.append(cat.iloc[i]["combined"])
        result.append("*****")
    return result

df["categories"] = df.apply(lambda x: convert_categories_index_to_text(x["category_ids"]), axis=1)

In [26]:
# QA

position = 4

print(df.iloc[position]["subject"])
print(df.iloc[position]["categories"])


This just IN! Color Her Happy and more. 
['electronics > video > video accessories > computer monitor accessories > color calibrators', '*****', 'vehicles & parts > vehicles > motor vehicles > off-road and all-terrain vehicles', '*****', 'food, beverages & tobacco > food items > condiments & sauces > sweet and sour sauces', '*****', 'health & beauty > health care > biometric monitors > fertility monitors and ovulation tests', '*****', 'health & beauty > personal care > hair care > hair color', '*****', 'vehicles & parts > vehicles > motor vehicles > off-road and all-terrain vehicles > atvs & utvs', '*****', 'vehicles & parts > vehicles > motor vehicles > off-road and all-terrain vehicles > go karts & dune buggies', '*****', 'health & beauty > personal care > hair care > hair color removers', '*****', 'arts & entertainment > hobbies & creative arts > arts & crafts > art & crafting materials > craft fasteners & closures > hook and loop fasteners', '*****', 'vehicles & parts > vehicle par

In [12]:
# next steps
# get this to run faster
# clean up categories and assess results
# how do we handle singular v.s. plural?
# should we remove stop words?
# what categories should we add? (shoes example)
# using more than just subject (e.g. full_text)
# using google vision to understand the email better (image recognition, OCR, etc)
### use this info to add new categories
# explore using different weights (would this make sense for us?)

In [94]:
# We tried using tfidf to look values up instead of our naive lookup approach
# This did not yield better results


# from sklearn.feature_extraction.text import TfidfVectorizer

# cat_tfidf = TfidfVectorizer()
# cat_tfidf = cat_tfidf.fit(cat.combined)
# cat_tfid_t = cat_tfidf.transform(cat.combined)

# test = df.iloc[2].subject
# print(test)

# test_vec = cat_tfidf.transform([test])

# from sklearn.metrics.pairwise import cosine_similarity
# cos = cosine_similarity(test_vec, cat_tfid_t)
# cat.iloc[cos.argmax()].combined

In [156]:
import string
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

In [148]:
# Get stopwords_set and punctuation set.
stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation)

In [151]:
all_words_in_subjects = []

for i, row in df.iterrows():
    word_tokens = word_tokenize(row.subject)
    filtered_sentence = [w.lower() for w in word_tokens if not w in stopwords and not w in punctuation and w.isalpha()]
    all_words_in_subjects.extend(filtered_sentence)
    
    word_tokens = word_tokenize(row.full_text)
    filtered_sentence = [w.lower() for w in word_tokens if not w in stopwords and not w in punctuation and w.isalpha()]
    all_words_in_subjects.extend(filtered_sentence)

In [155]:
# basic word count
from collections import Counter
counts = Counter(all_words_in_subjects)
counts.most_common()

[('the', 149),
 ('share', 137),
 ('email', 122),
 ('free', 99),
 ('new', 95),
 ('shipping', 94),
 ('i', 72),
 ('valid', 66),
 ('shop', 63),
 ('us', 62),
 ('more', 61),
 ('you', 57),
 ('unsubscribe', 54),
 ('stores', 46),
 ('online', 46),
 ('now', 45),
 ('click', 45),
 ('see', 44),
 ('view', 44),
 ('vox', 43),
 ('use', 43),
 ('this', 40),
 ('policy', 40),
 ('get', 40),
 ('offer', 40),
 ('may', 40),
 ('please', 40),
 ('to', 39),
 ('read', 39),
 ('privacy', 39),
 ('one', 38),
 ('your', 38),
 ('not', 35),
 ('address', 35),
 ('facebook', 35),
 ('news', 34),
 ('code', 33),
 ('sales', 33),
 ('place', 33),
 ('would', 33),
 ('orders', 32),
 ('women', 32),
 ('gift', 31),
 ('could', 31),
 ('price', 31),
 ('card', 29),
 ('change', 29),
 ('find', 29),
 ('in', 29),
 ('day', 29),
 ('like', 28),
 ('if', 27),
 ('total', 27),
 ('twitter', 26),
 ('make', 26),
 ('de', 26),
 ('no', 26),
 ('purchases', 25),
 ('terms', 25),
 ('marketing', 25),
 ('lowest', 24),
 ('all', 24),
 ('march', 24),
 ('know', 24),
 ('

In [161]:
syns = wordnet.synsets('stack')

In [177]:
for s in syns:
    for l in s.lemmas():
        print(str(l.name()) + " " + str(l.count()))
        break

stack 2
batch 0
push-down_list 0
smokestack 0
push-down_storage 0
stack 2
stack 0
stack 0


In [189]:
i = 0
syns[i].lemmas()[i].name()

'stack'

In [ ]:
## Next steps
# Get scraper working: grab top 20 words on 5 websites
# Get word freq and see if anything stands out
## Alternatives
# Team manually builds the word list
# 